In [2]:
# code adapted from playcrypt


def exp_mod(a, n, N):
	r = 1
	for i in range(n.bit_length() - 1, -1, -1):
		r = ((r*r) * (a if (n >> i) & 0x1 else 1)) % N

	return r % N

def prime_between(s, e):
    candidate = randint(int(s), int(e))

    while not (is_prime(candidate)):
        candidate = randint(int(s), int(e))

    return candidate

def modinv(a, m):
    """
    Computes the modular inverse of a mod m.
    http://stackoverflow.com/questions/4798654/modular-multiplicative-inverse-function-in-python

    :param a: a in a^-1 mod m
    :param m: m in a^-1 mod m
    :return: a^-1 mod m or None if no inverse exists
    """
    g, x, y = xgcd(a, m)
    if g != 1:
        return None
    else:
        return x % m

def in_Z_N_star(a, N):
    """
    Determines whether a is in Z_n^*.

    :param a: number to be checked
    :param N: N
    :return: Boolean
    """
    if a < 0 or a>=N:
        return False
    return(xgcd(a,N)[0] == 1)

def random_Z_N_star(N):
    """
    Returns the a random element in Z_N^*.

    :param N: N
    :return: random element
    """
    candidate = randint(1, N-1)

    while not (in_Z_N_star(candidate, N)):
        candidate = randint(1, N-1)

    return candidate

def k_rsa(k):
    # Generate N, p, q
    while True:
        # Let p be a prime such that 2**(k/2-1) <= p <= 2**(k/2)
        p = prime_between(2**(int(k/2-1)), 2**(int(k/2)))
        # Let q be a prime such that 2**(k-1) <= p*q <= 2**k
        left = 2**(k-1) // p
        right = 2**k // p
        q = prime_between(left, right)
        N = p * q
        if p != q and 2**(k-1) <= N and N <= 2**k:
            break
    # Generate e, d
    phi = (p - 1) * (q - 1) # phi(N)
    e = random_Z_N_star(phi)
    d = modinv(e, phi)
    return (N, p, q, e, d)

In [22]:
# https://github.com/asanso/CryptoWithSageMath/blob/master/rsa.sage

def RSA_e(bits, e):
    # RSA generation, except you can specify e
    if e in [0, 1]:
        raise ValueError("e cannot be 0, 1")
    if e >= 2**bits:
        raise ValueError("e cannot be geater than 2^bits")
    d = None
    i = 0
    while d == None:
        if i > 1000:
            raise ValueError('taking too long, check for errors in input...')
        p = random_prime(2**(bits)//2, proof = True)
        q = random_prime(2**(bits)//2, proof = True)
        N = p * q
        phi = (p - 1)*(q - 1)
        d = modinv(e, phi)
        i = i + 1
    return (N, p, q, e, d)

def k_tfrm(Key):
    # generate key transform variable
    (N, p, q, e, d) = Key
    phi = (p-1)*(q-1)
    g = 2
    i = 0
    while g != 1:
        if i > 100:
            raise ValueError('taking too long, check for errors in input...')
        # parameter from the paper
        t = randint(1, N)
        g = xgcd(t, phi)[0]
        i = i + 1
    e_s = e * t % phi
    d_s =  xgcd(e_s, phi)[1] % phi
    return (t, e_s, d_s)

def key_pub(bits, e_p=3):
    # defining a publicly available exponent (i.e. want e_p to be small)
    key = RSA_e(bits, e_p)
    tfrm = k_tfrm(key)
    return (key, tfrm)

def key_sec(bits, e_s=2**16 + 1):
    # defining using a secret exponent (i.e. only e_s is supported by HSM)
    (N, p, q, e_s, d_s) = RSA_e(bits, e_s)
    (t, e_p, d_p) = k_tfrm((N, p, q, e_s, d_s))
    tfrm = (modinv(t, (p - 1)*(q - 1)), e_s, d_s)
    key = (N, p, q, e_p, d_p)
    return (key, tfrm)

def key_ran(bits):
    # generate RSA key and transform using random choice of e
    key = k_rsa(bits)
    tfrm = k_tfrm(key)
    return (key, tfrm)

def test(key, tfrm):
    # test whether the implementation of exponent transform is correct
    (N, p, q, e_p, d_p) = key
    (t, e_s, d_s) = tfrm
    value = random_prime(2**bits)

    sign = exp_mod(value, d_s, N)

    t_sign = exp_mod(sign, t, N)

    p_sign = exp_mod(value, d_p, N)

    print('value is: ', value)
    print('transform sign is: ', t_sign)
    print('transform sign is: ', p_sign)


In [23]:
bits = 8

std = 2**16 + 1

((N, p, q, e_p, d_p),(t, e_s, d_s)) = key_ran(512)

if False:
    print ("N:", N, 'φ(N):', (p-1)*(q-1), "p: ", p, " q:", q)
    print('d_p: ', d_p, 'e_p: ', e_p)
    print('d_s: ', d_s, 'e_s: ', e_s)
    print('t: ', t)

k_estimate = round(((e_p * t)) / (N - round(N**(1/2)) + 1))
print('k estimate:', k_estimate)

k_real = round(((e_p * t) - e_s) / (N - (p+q) + 1))
print('k real val:', k_real)

print('ratio:', n(k_real / k_estimate))
print('diff:', k_estimate - k_real)
print('log:', n(log(abs(k_estimate - k_real))/log(2)))

k estimate: 5852454599744729683001237840501487923375985245111026763098434073966955357989516882558506153277351674971318766208344055528176042138938689007264157822981084
k real val: 5852454599744729683001237840501487923375985245111026763098434073966955357989584910593892906695856379823730105390308927227830642211311607539687439237139465
ratio: 1.00000000000000
diff: -68028035386753418504704852411339181964871699654600072372918532423281414158381
log: 255.232664636890
